In [ ]:
#we will instead use the examples in the full chrombpnet repo. This is saved in chrombpnet test
#it has 3 replicates of the bam files, and uses a separate reference genome, but we can compare it to our old reference genome
# samtools merge -f merged_unsorted.bam rep1.bam  rep2.bam  rep3.bam
# samtools sort -@4 merged_unsorted.bam -o merged.bam
# samtools index merged.bam


#chrombpnet prep splits -c hg38.chrom.subset.sizes -tcr chr11 chr12 -vcr chr8 chr10 -op splits/fold_0
#this command isn't included, so we just manually made this instead

#also one part where we have to implment it...
#chrombpnet prep nonpeaks -g ~/chrombpnet_tutorial/data/downloads/hg38.fa -p ~/chrombpnet_tutorial/data/peaks_no_blacklist.bed -c  ~/chrombpnet_tutorial/data/downloads/hg38.chrom.sizes -fl ~/chrombpnet_tutorial/data/splits/fold_0.json -br ~/chrombpnet_tutorial/data/downloads/blacklist.bed.gz -o ~/chrombpnet_tutorial/data/output
#	args = parsers.read_parser()

#made a simple test to see what args is

# python utils/test_args.py prep nonpeaks -g hg38.fa -p peaks_no_blacklist.bed -c  hg38.chrom.sizes -fl fold_0.json -br blacklist.bed.gz -o /data/output
# Namespace(cmd='prep', cmd_prep='nonpeaks', genome='hg38.fa', output_prefix='/data/output', peaks='peaks_no_blacklist.bed', chrom_sizes='hg38.chrom.sizes', chr_fold_path='fold_0.json', inputlen=2114, stride=1000, neg_to_pos_ratio_train=2, blacklist_regions='blacklist.bed.gz', seed=1234)
#can look at the parser to see what the commands like -g are, -g is genome, -p is peaks, etc.
#it's the section right after general training args!
#but we don't actually really need it, can just copy the functions over!
# so we got all of it, let's see what happens if we call that?

# python utils/prep_nonpeaks.py prep nonpeaks -g hg38.fa -p peaks_no_blacklist.bed -c  hg38.chrom.sizes -fl splits/fold_0.json -br blacklist.bed.gz -o output

#this has an issue with the args.input
#it was actually an issue with the data, we somehow downloaded the overlap file incorrectly, we'll make this process much better later on and use the built in pipelines, but for now,need to understand what it's doing! 


In [ ]:
#now we can view the final files, we have this merged and index bam file, let's view this
#he actually also has an easy way to get the gc content, which is pretty easy to use
#bpnet negatives -i <peaks>.bed -f <fasta>.fa -b <bigwig>.bw -o matched_loci.bed -l 0.02 -w 2114 -v
#wait, we don't have a bigwig file??

#but we have the negatives, so it doesn't matter
#now let's create the json for training, we need

#we need to create the bigwig file as well
#bedtools genomecov -5 -bg -g hg38.chrom.sizes -ibam merged.bam | sort -k1,1 -k2,2n > data.bedGraph
# ./bedGraphToBigWig data.bedGraph <your genome>.chrom.sizes data.bw

In [ ]:
#now let's do this again with actual chrombpnet package
# first mamba install all the things they say and tensorflow-gpu=2.8.0
# chrombpnet bias pipeline \
#         -ibam merged.bam \
#         -d "ATAC" \
#         -g hg38.fa \
#         -c hg38.chrom.sizes \
#         -p peaks_no_blacklist.bed \
#         -n output_negatives.bed \
#         -fl splits/fold_0.json \
#         -b 0.5 \
#         -o bias_model/ \
#         -fp k562 \

#that's hwo we'll train this bias model, then we can do the full model

#then we train the actual model

# chrombpnet pipeline \
#         -ibam merged.bam \
#         -d "ATAC" \
#         -g hg38.fa \
#         -c hg38.chrom.sizes \
#         -p peaks_no_blacklist.bed \
#         -n output_negatives.bed \
#         -fl splits/fold_0.json \
#         -b bias_model/models/k562_bias.h5 \
#         -o chrombpnet_model/